**1) Environment Setup, Install, and Load Dependencies**


Cell re-establishes the environment, installs the necessary package, and loads all the crucial foreign key IDs needed to link the final activity records (listings, appointments, transactions, etc.) back to Agents, Clients, and Properties.

In [2]:
### 1. Setup, Imports, and Load Foreign Key IDs ###

# Reinstall Faker as the environment resets in a new notebook
%pip install faker

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import csv
import random
import json
import os
from datetime import datetime, timedelta
from faker import Faker

# --- CONFIGURATION (Must match preceding Notebooks) ---
fake = Faker()
Faker.seed(42)
random.seed(42)

# Set the OUTPUT DIRECTORY (MUST match your path)
output_dir = '/content/drive/MyDrive/SQL Project (Group 2)/Simulating Data (Maya)/Simulated Data Files'
os.makedirs(output_dir, exist_ok=True)

# --- Data Utility Functions ---

def load_ids(filename):
    """Loads a single column of integer IDs from a specified CSV file."""
    ids = []
    filepath = os.path.join(output_dir, filename)
    try:
        with open(filepath, 'r', newline='') as file:
            reader = csv.reader(file)
            next(reader)  # Skip header
            for row in reader:
                ids.append(int(row[0]))
    except FileNotFoundError:
        print(f"Error: {filename} not found. Please ensure preceding notebooks were run.")
    return ids

def write_csv(filename, headers, data_rows):
    """Writes data to a CSV file, converting empty strings to None."""
    filepath = os.path.join(output_dir, filename)
    with open(filepath, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        clean_rows = [[(val if val != '' else None) for val in row] for row in data_rows]
        writer.writerows(clean_rows)

# --- Load Necessary Foreign Keys ---
agent_ids = load_ids('agent_ids.csv')
client_ids = load_ids('client_ids.csv')
property_ids = load_ids('property_ids.csv')
listing_ids = list(range(1, 101))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.1 MB/s eta 0:00:00
Mounted at /content/drive


**2) Generate Transactional and Activity Data**

Generates all activity-based tables: **Listings** (Level 5), and the six subsequent **Transactional Tables** (Level 6).

* **Listings (Fixed Logic):** The code now correctly bounds the `closed_date` to prevent the "empty range" error while ensuring the closing date is always after the listing date.
* **Transactions:** The `transactions` table omits `office_id` in the CSV, relying on your PostgreSQL trigger to populate it automatically.

**3) Generate Listings and Activities**


Cell executes the logic to generate the remaining five tables and saves the CSVs.

In [3]:
### Generate Listings and Activities ###

# --- 8. Generate Listings (100) ---
listings = []
for i in range(100):
    property_id = property_ids[i]
    agent_id = random.choice(agent_ids)
    list_price = round(random.uniform(300000, 3000000), 2)

    status_id = random.choices([1, 2, 3, 4, 5], weights=[40, 10, 35, 10, 5])[0]

    closed_date = ''

    if status_id in [3, 4]: # Status is Sold (3) or Rented (4)
        # 1. Generate the guaranteed closed date (max date is today)
        closed_date_obj = fake.date_between(start_date='-1y', end_date='today')
        closed_date = closed_date_obj.strftime('%Y-%m-%d')

        # 2. Set the cap for the listed date to 1 day before closing
        max_listed_date_obj = closed_date_obj - timedelta(days=1)

        # 3. Set the absolute minimum historical date (2 years ago)
        min_listed_date = datetime.now().date() - timedelta(days=365 * 2)

        # Generate listed date within the chronologically valid range
        if max_listed_date_obj < min_listed_date:
            # If the closing date is ancient, use the closing date minus 1 day as the listed date
            listed_date_obj = max_listed_date_obj
        else:
            # Generate random date between historical start and the closing cap.
            listed_date_obj = fake.date_between(start_date=min_listed_date, end_date=max_listed_date_obj)

        listed_date = listed_date_obj.strftime('%Y-%m-%d')

    else: # Active, Pending, Withdrawn (status_id 1, 2, 5) - no chronological constraint conflict
        listed_date_obj = fake.date_between(start_date='-2y', end_date='today')
        listed_date = listed_date_obj.strftime('%Y-%m-%d')

    listings.append([i+1, property_id, agent_id, list_price, status_id, listed_date, closed_date])

write_csv('listings.csv',
    ['listing_id', 'property_id', 'agent_id', 'list_price', 'listing_status_id', 'listed_date', 'closed_date'],
    listings)


# --- 9. Generate Appointments (100) ---
appointments = []
for i in range(100):
    appt_type = random.randint(1, 4)
    appt_datetime = fake.date_time_between(start_date='-6m', end_date='now').strftime('%Y-%m-%d %H:%M:%S')
    listing_id = random.choice(listing_ids)
    client_id = random.choice(client_ids)
    outcome_id = random.randint(1, 4) if random.random() < 0.8 else ''
    notes_options_by_outcome = {
        1: "Client attended, gathering feedback.", 2: "No show, attempting to reschedule.",
        3: "Client loved the property, drafting an offer.", 4: "Requires follow-up on financing.",
        '': "Outcome pending or appointment not yet complete."
    }
    notes = notes_options_by_outcome.get(outcome_id, notes_options_by_outcome[''])

    appointments.append([i+1, appt_type, appt_datetime, listing_id, client_id, outcome_id, notes])

write_csv('appointments.csv',
    ['appointment_id', 'appointment_type_id', 'appointment_datetime', 'listing_id', 'client_id',
     'appointment_outcome_id', 'notes'],
    appointments)


# --- 10. Generate Offers (120) ---
offers = []
for i in range(120):
    listing_id = random.choice(listing_ids)
    buyer_client_id = random.choice(client_ids)
    base_price = random.uniform(300000, 3000000)
    offer_amount = round(base_price * random.uniform(0.90, 1.05), 2)
    offer_date = fake.date_between(start_date='-1y', end_date='today').strftime('%Y-%m-%d')
    status_id = random.choices([1, 2, 3, 4, 5, 6], weights=[20, 25, 30, 10, 10, 5])[0]
    notes = random.choice(["Competitive offer with quick closing", "Subject to financing approval", "All cash offer", "Contingent on selling current home", ''])

    offers.append([i+1, listing_id, buyer_client_id, offer_amount, offer_date, status_id, notes])

write_csv('offers.csv',
    ['offer_id', 'listing_id', 'buyer_client_id', 'offer_amount', 'offer_date', 'offer_status_id', 'notes'],
    offers)


# --- 11. Generate Transactions (100) ---
transactions = []
for i in range(100):
    listing_id = listing_ids[i]
    buyer_client_id = random.choice(client_ids)
    seller_client_id = random.choice(client_ids)
    agent_id = random.choice(agent_ids)
    tx_type = random.choices(['Sale', 'Rental'], weights=[80, 20])[0]

    close_price = round(random.uniform(300000, 3000000), 2) if tx_type == 'Sale' else round(random.uniform(1500, 8000), 2)
    commission_pct = random.uniform(0.02, 0.06)
    commission_amount = round(close_price * commission_pct, 2)
    close_date = fake.date_between(start_date='-1y', end_date='today').strftime('%Y-%m-%d')

    transactions.append([i+1, listing_id, buyer_client_id, seller_client_id, agent_id,
                        tx_type, close_price, round(commission_pct, 4), commission_amount, close_date])

write_csv('transactions.csv',
    ['transaction_id', 'listing_id', 'buyer_client_id', 'seller_client_id', 'agent_id',
     'transaction_type', 'close_price', 'commission_pct', 'commission_amount', 'close_date'],
    transactions)


# --- 12. Generate Marketing Campaigns (150) ---
campaigns = []
channels = ['website', 'social media', 'email', 'print', '3rd-party portal']

for i in range(150):
    listing_id = random.choice(listing_ids)
    channel = random.choice(channels)
    start_date = fake.date_between(start_date='-1y', end_date='today').strftime('%Y-%m-%d')

    end_date = ''
    if random.random() < 0.7:
        min_end_date = datetime.strptime(start_date, '%Y-%m-%d').date() + timedelta(days=1)
        max_end_date = datetime.now().date()

        if min_end_date <= max_end_date:
            end_date_obj = fake.date_between(start_date=min_end_date, end_date=max_end_date)
            end_date = end_date_obj.strftime('%Y-%m-%d')

    cost = round(random.uniform(100, 5000), 2)
    impressions = random.randint(1000, 100000)
    clicks = random.randint(10, int(impressions * 0.05))

    campaigns.append([i+1, listing_id, channel, start_date, end_date, cost, impressions, clicks])

write_csv('marketing_campaigns.csv',
    ['marketing_campaign_id', 'listing_id', 'channel', 'start_date', 'end_date', 'cost', 'impressions', 'clicks'],
    campaigns)

print("SUCCESS: All temporal errors resolved and CSVs regenerated.")

SUCCESS: All temporal errors resolved and CSVs regenerated.


**4) Checkpoint 4 Completion**

All required CSV files for the relational database schema have been generated and saved to Google Drive folder:
* `addresses.csv`
* `offices.csv`
* `users.csv`
* `agents.csv`
* `clients.csv`
* `client_preferences.csv`
* `properties.csv`
* `listings.csv`
* `appointments.csv`
* `offers.csv`
* `transactions.csv`
* `marketing_campaigns.csv`
